In [3]:
import pandas as pd
import talib as TA
import pandas_ta as ta
from backtesting import Strategy, Backtest

READ & CLEAN THE DATA SET

In [2]:
df = pd.read_csv('data_sets/data_set_EURUSD_Candlestick_15_M_BID_31.01.2019-30.01.2022.csv')
df = df[df['Volume']!=0]
df.isna().sum()
df.reset_index(drop=True, inplace=True)
df.tail()

,Local time,Open,High,Low,Close,Volume
74758,30.01.2022 22:45:00.000 GMT-0500,1.11619,1.11629,1.11607,1.11608,709.03
74759,30.01.2022 23:00:00.000 GMT-0500,1.11609,1.11634,1.11602,1.11607,377.49
74760,30.01.2022 23:15:00.000 GMT-0500,1.11606,1.11621,1.11603,1.11615,267.24
74761,30.01.2022 23:30:00.000 GMT-0500,1.11614,1.11636,1.11608,1.11633,261.17
74762,30.01.2022 23:45:00.000 GMT-0500,1.11632,1.11636,1.11616,1.11616,242.71


CALCULATION OF ATR

In [4]:
df['ATR']= df.ta.atr()
df.tail()

,Local time,Open,High,Low,Close,Volume,ATR
74758,30.01.2022 22:45:00.000 GMT-0500,1.11619,1.11629,1.11607,1.11608,709.03,0.000465
74759,30.01.2022 23:00:00.000 GMT-0500,1.11609,1.11634,1.11602,1.11607,377.49,0.000455
74760,30.01.2022 23:15:00.000 GMT-0500,1.11606,1.11621,1.11603,1.11615,267.24,0.000435
74761,30.01.2022 23:30:00.000 GMT-0500,1.11614,1.11636,1.11608,1.11633,261.17,0.000424
74762,30.01.2022 23:45:00.000 GMT-0500,1.11632,1.11636,1.11616,1.11616,242.71,0.000408


BACKTEST ATR RELATED SL&TP

In [5]:
class MyStrat(Strategy):
    initsize = 0.3
    mysize = initsize
    ema_window = 200
    rsi_window = 3
    atr_window = 14
    
    def init(self):
        super().init()
        self.ema = self.I(TA.EMA, self.data.Close, self.ema_window)
        self.rsi = self.I(TA.RSI, self.data.Close, self.rsi_window)
        self.atr = self.I(TA.ATR, self.data.High, self.data.Low, self.data.Close, self.atr_window)        

    def next(self):
        super().next()
        slatr = 1.2*self.data.ATR[-1]
        TPSLRatio = 1.5
        price = self.data.Close[-1] 
        
        if self.position:
            self.position.close()
            
        if self.data.Close >= self.ema and self.rsi <= 10:
            sl1 = price - slatr
            tp1 = price + slatr*TPSLRatio
            
            self.buy(sl=sl1, tp=tp1, size = self.mysize)
            
        if self.data.Close <= self.ema and self.rsi >= 90:
            sl2 = price + slatr
            tp2 = price - slatr*TPSLRatio
            
            self.sell(sl=sl2, tp=tp2, size = self.mysize)

bt = Backtest(df, MyStrat, cash=500, margin=1/50, commission=.00)
stat = bt.run()
stat

/tmp/ipykernel_16775/3536457354.py:35: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, MyStrat, cash=500, margin=1/50, commission=.00)


Start                                     0.0
End                                   74762.0
Duration                              74762.0
Exposure Time [%]                    3.865548
Equity Final [$]                   1275.11767
Equity Peak [$]                   1310.613895
Return [%]                         155.023534
Buy & Hold Return [%]               -2.978043
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -12.285725
Avg. Drawdown [%]                   -0.903907
Max. Drawdown Duration                13869.0
Avg. Drawdown Duration             459.985612
# Trades                               1888.0
Win Rate [%]                         54.07839
Best Trade [%]                        0.47816
Worst Trade [%]                     -0.186084
Avg. Trade [%]                    

OPTIMIZATION

In [6]:
stats = bt.optimize(
    ema_window = range(50, 200),
    rsi_window = range(2, 10),
    maximize = 'Equity Final [$]',
)
print(stats)
print(stats._MARSI)
print(stats._strategy)

/home/samueltg92/miniconda3/envs/TRADING/lib/python3.9/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 1200 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/12 [00:00<?, ?it/s]